In [205]:
import csv
import numpy as np
import cv2

records  = []

directory = "object-dataset/"
cardirectory = "object-dataset-cars/"
notcardirectory = "object-dataset-notcars/"

with open(directory + "labels.csv", "r") as csvfile:
    reader = csv.reader(csvfile, delimiter=" ", quotechar='"')
    for line in reader:
        time = np.int64(line[0].split(".")[0])
        records.append([time] + line)

In [151]:
records = np.sort(records, axis=0)

    

In [153]:
prev_time = 0
time_break = 0
series = []
series_count = 0
cars = []
for r in records:
    if r[-1] == 'car':
        cars.append(r)
        diff = (r[0] - prev_time) * (1/1000000000.0)
        if diff > 3:
            series.append(series_count)
            time_break += 1
            series_count = 0
        series_count +=1
        prev_time = r[0]

In [133]:
print(time_break, len(cars))

66 60788


In [85]:
print(cars[0])

4.75134839365


In [225]:
def make_bbox_square(f,shape):
    f[2] = int(f[2])
    f[3] = int(f[3])
    f[4] = int(f[4])
    f[5] = int(f[5])
    
    width = f[4] - f[2]
    height = f[5] - f[3]
    plusminus = int(np.absolute((width-height)/2))
    if width > height:
        box = ((f[2], f[4]), np.clip(shape[0], 0, (f[3]-plusminus, f[5]+plusminus)))
    else:
        box = (np.clip(shape[1], 0, (f[2]-plusminus, f[4]+plusminus)), (f[3], f[5]))
    return box
    
def validate_box(box, heat):
    shape = heat.shape
    if box[0][0] < 0 or box[1][0] < 0:
        return False
    if box[0][1] > shape[1] or box[1][1] > shape[0]:
        return False
    if np.sum(heat[box[1][0]:box[1][1], box[0][0]:box[0][1]]) > 0:
        return False
    return True
    
def get_neighboring_box(box, heat, rand):
    dim = box[0][1]-box[0][0]
    
    if rand == 0:
        #left
        new = ((box[0][0]-dim, box[0][1]-dim), box[1])
    elif rand == 1:
        #up
        new = (box[0], (box[1][0]-dim, box[1][1]-dim))
    elif rand == 2:
        #right
        new = ((box[0][0]+dim, box[0][1]+dim), box[1])
    elif rand == 3:
        #down
        new = (box[1], (box[1][0]+dim, box[1][1]+dim))
    if validate_box(new, heat):
        return new
        
def add_heat(heatmap, bbox_list):
    for box in bbox_list:
        heatmap[box[1][0]:box[1][1], box[0][0]:box[0][1]] += 1
    return heatmap
        
def pull_cars_from_frame(frame_cars):
    rand = [0,1,2,3]
    bboxes = []
    antiboxes = []
    img = cv2.imread(directory + frame_cars[0][1])
    shape = img.shape
    heat = np.zeros_like(img[:,:,:]).astype(np.float)
    for i in range(len(frame_cars)):
        f = frame_cars[i]
        bbox = make_bbox_square(f, shape)
        bboxes.append(bbox)
    heatmap = add_heat(heat, bboxes)
    for box in bboxes:
        np.random.shuffle(rand)
        for r in rand:
            anti = get_neighboring_box(box, heatmap, r)
            if anti:
                antiboxes.append(anti)
                break
    for i in range(len(bboxes)):
        box = bboxes[i]
        out = cv2.resize(img[box[1][0]:box[1][1], box[0][0]:box[0][1]], (64,64))
        cv2.imwrite(cardirectory + frame_cars[0][1] + str(i) + ".png", out)
    for i in range(len(antiboxes)):
        box = antiboxes[i]
        out = cv2.resize(img[box[1][0]:box[1][1], box[0][0]:box[0][1]], (64,64))
        cv2.imwrite(notcardirectory + frame_cars[0][1] + str(i) + ".png", out)
    
    
        

car_count = 0
current_time = cars[0][0]
frame_cars = []
count = 0
for c in cars:
    time_diff = (c[0] - current_time) * (1/1000000000.0)
    if time_diff > 2:
        pull_cars_from_frame(frame_cars)
        frame_cars = [c]
        current_time = c[0]
        count += 1
    elif c[0] == current_time:
        if c[6] == '0':
            car_count += 1
            frame_cars.append(c)
        

In [218]:
np.clip(2000,0,((-98, 186), (468, 752)))

array([[  0, 186],
       [468, 752]])

In [226]:
[1] + [2]


[1, 2]